In [10]:
import pandas as pd
import json
import os
from tqdm import tqdm
import synthpops as sps
import numpy as np
import covasim as cv
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import pylab as pl
import sciris as sc
import scipy as sp
import optuna as op
from datetime import datetime, date
import random
import time

from autocalibration_covasim.functions_total import *
import warnings
warnings.filterwarnings("ignore")

import autocalibration_covasim.calibration_total as st
from autocalibration_covasim.calibration_total import model

# Создание популяции

### Подготовка данных

In [ ]:
schools_path = './data/chelyabinsk_25/schools.json'
people_path = './data/chelyabinsk_25/people.txt'
households_path = './data/chelyabinsk_25/households.txt'
pop_path = './data/chelyabinsk_25/pop_covasim.json'

In [ ]:
school = json.load(open(os.path.expanduser(schools_path)))
people = pd.read_csv(people_path, sep='\t')
households = pd.read_csv(households_path, sep='\t')

### Создание словаря id: age

In [ ]:
people_by_age = {}
for i in tqdm(range(101)):
    people_by_age[i]=sorted(list(people[people.age==i]['sp_id']))

In [ ]:
age_by_id = {}
for i in people_by_age:
    for j in people_by_age[i]:
        age_by_id[j]=i

### Создание списка списков жителей одного дома

In [ ]:
households = []
for i in tqdm(list(people.groupby('sp_hh_id').size().index)):
    households.append(list(people[people.sp_hh_id==i]['sp_id']))

### Создание списка списков с учениками одной школы

In [ ]:
schools = []
for i in tqdm(list(people[(people.age<18)&(people.work_id!='X')].groupby('work_id').size().index)):
    schools.append(list(people[(people.work_id==i)&(people.age<18)]['sp_id']))

### Создание списка списков с рабочими из одного рабочего места

In [ ]:
works = []
for i in tqdm(list(people[(people.age>=18)&(people.work_id!='X')].groupby('work_id').size().index)):
    works.append(list(people[(people.work_id==i)&(people.age>=18)]['sp_id']))

### Создание списка списков с учителями из одной школы

In [ ]:
teachers = []
for i in range(len(schools)):
    teachers.append([])#schools[i][0]])
    #del schools[i][0]

### Генерация словаря популяции

In [ ]:
popdict = sps.contact_networks.make_contacts(
    sps.pop.Pop, 
    age_by_uid=age_by_id, 
    homes_by_uids=households, 
    students_by_uid_lists=schools,
    workplace_by_uid_lists=works,
    teachers_by_uid_lists=teachers,
    average_class_size=100000,
)

### Конвертация в json

In [ ]:
def myconverter(obj):
        if isinstance(obj, np.int64):
            return int(obj)
        if isinstance(obj, set):
            return list(obj)
        raise TypeError

In [ ]:
with open(pop_path, "w", encoding="utf-8") as file:
    json.dump(popdict, file, default=myconverter)

# Загрузка популяции из json

In [ ]:
pop_path = './data/chelyabinsk_25/pop_covasim.json'

In [ ]:
jsonpop = json.load(open(os.path.expanduser(pop_path)))

popdict = {}
for i in jsonpop:
    popdict[int(i)] = jsonpop[i]
    for j in ['H', 'S', 'W', 'C']:
        popdict[int(i)]['contacts'][j] = set(jsonpop[i]['contacts'][j])

# Работа с ковасимом

### Создание симуляции с заданной популяцией

In [ ]:
pars = {'pop_size': 40797,
 'pop_infected': 98,
 'pop_type': 'synthpops',
 'location': 'Chelyabinsk',
 'n_days': 150,
 'rand_seed': 1,
 'verbose': 0,
 'pop_scale': 1,
 'scaled_pop': None,
 'rescale': False,
 'rescale_threshold': 1,
 'rescale_factor': 1,
 'frac_susceptible': 0.78,
 'contacts': {'h': 1.57, 's': 8.5, 'w': 8.5, 'c': 0, 'l': 10},
 'dynam_layer': {'h': 0, 's': 0, 'w': 0, 'c': 0, 'l': 0},
 'beta_layer': {'h': 3.0, 's': 0.6, 'w': 0.6, 'c': 0, 'l': 1.5},
 'beta_dist': {'dist': 'neg_binomial',
  'par1': 1.0,
  'par2': 0.45,
  'step': 0.01},
 'viral_dist': {'frac_time': 0.3, 'load_ratio': 2, 'high_cap': 4},
 'beta': 0.23451319609633497,
 'asymp_factor': 1.0,
 'n_imports': 0,
 'n_variants': 1,
 'use_waning': False,

 'rel_imm_symp': {'asymp': 0.85, 'mild': 1, 'severe': 1.5},
 'immunity': None,
 'trans_redux': 0.59,
 'rel_beta': 1.0,
 'interventions': [cv.test_num(daily_tests=40797, symp_test=100.0, quar_test=1.0, quar_policy=None, subtarget=None, 
                               ili_prev=None, sensitivity=1.0, loss_prob=0, test_delay=0, start_day=0, end_day=None, 
                               swab_delay=None)],
 'rel_symp_prob': 1.0,
 'rel_severe_prob': 1.0,
 'rel_crit_prob': 0,
 'rel_death_prob': 0,
 'prog_by_age': False,
 'prognoses': {'age_cutoffs': np.array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90]),
      'sus_ORs':  np.array([0.34, 0.67, 1.  , 1.  , 1.  , 1.  , 1.24, 1.47, 1.47, 1.47]),
      'trans_ORs':  np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
      'comorbidities':  np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
      'symp_probs':  np.array([0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.9 ]),
      'severe_probs':  np.array([0.001, 0.003, 0.012, 0.032, 0.049, 0.102, 0.166, 0.243, 0.273,
             0.273]),
      'crit_probs':  np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
      'death_probs':  np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])},
 'iso_factor': {'h': 1, 's': 1, 'w': 1, 'c': 1, 'l': 1},
 'quar_factor': {'h': 1, 's': 1, 'w': 1, 'c': 1, 'l': 1},
 'quar_period': 0,
 'analyzers': [],
 'timelimit': None,
 'stopping_func': None,
 'n_beds_hosp': None,
 'n_beds_icu': None,
 'no_hosp_factor': 1,
 'no_icu_factor': 1,
 'vaccine_pars': {},
 'vaccine_map': {},
 'variants': [],
 'variant_map': {0: 'wild'},
 'variant_pars': {'wild': {'rel_beta': 1.0,
   'rel_symp_prob': 1.0,
   'rel_severe_prob': 1.0,
   'rel_crit_prob': 0,
   'rel_death_prob': 0}}}


sim = cv.Sim(pars, interventions=cv.test_num(daily_tests=len(popdict)), datafile='./data/chelyabinsk_25/Chelyabinsk_dataset.csv')
sim.popdict = cv.make_synthpop(sim, popdict=popdict)

### Калибровка

In [ ]:
calib_pars = dict(
    beta           = [0.23451319609633497, 0.1, 0.50],
    pop_infected   = [10, 1, 100]
)
calib = sim.calibrate(calib_pars=calib_pars, total_trials=100)

In [ ]:
calib.plot_sims(to_plot=['new_infections','cum_infections'])

### Автокалибровка

In [ ]:
location='Chelyabinsk'
pop_location=40797
cal_keys=['n_infectious', 'new_diagnoses', 'cum_diagnoses'] # ключи для калибровки
cal_values=[1, 2, 3] # веса для параметров
pop_inf=[98,1,150] # количество первоначально зараженных: лучшее, мин, макс

data_csv="./data/chelyabinsk_25/Chelyabinsk_dataset.csv"


df1=pd.read_csv(data_csv,index_col=0,parse_dates=True)

# fill unknown tests
df1.new_tests=past_extr(df=df1,series=df1.new_tests,n=df1['new_tests'].isna().sum())
df1=smooth_pd(df1)
df1['date']=df1.index

# define start day and last day, bounds_of_periods for calibration
start_day=cv.date(df1.index[0].to_pydatetime().date())
last_day=cv.date(df1.index[-1].to_pydatetime().date())
bounds_of_periods=bounds_of_per(start_day,last_day)
bounds_of_periods.append(cv.date(last_day, as_date=False))
                         
# parameters for plotting
figsize=(18,5)
fontsize=18
fontsize_legend=15
linewidth=2
ticks=50
rotation=0
dateformat='%Y.%m.%d'

In [ ]:
# import calibrated parameters for Novosibirsk
calibrated_params=open('./data/chelyabinsk_25/p_autocal_total.json','r')
p=json.load(calibrated_params)


# define beta_changes and beta_days from calibrated parameters in list 'p'
b_days=[]
b_changes=[]
for i in range(len(p)):
    b_days.append(p[i][f'beta_day_{i+1}'])
    b_changes.append(p[i][f'beta_change_{i+1}'])
b_days=list(map(int, b_days))


# dates of beta_change
b_change_model=cv.date(b_days,start_date=start_day, as_date=False)

In [ ]:
# do autocalibration

# create an empty list for calibrated parameters
p=[]

for i in range(len(bounds_of_periods)):
    print('p before calibration', p)
    sim,p=calibration_process(param=p, datafile=df1, location=location, pop_location=pop_location, start_day=start_day,
                              end_day=bounds_of_periods[i], cal_keys=cal_keys, cal_values=cal_values, pop_inf=pop_inf,
                              #n_runs=n_runs, n_workers=n_workers,
                              popdict=popdict)
    print('p after calibration', p)

with open('p_autocal_total.json', 'w') as jsonfile:
    json.dump(p, jsonfile)

In [ ]:
sim=run_model(p=p, location=location, pop_location=pop_location, start_day=start_day, end_day=last_day, 
              b_days=b_days, b_changes=b_changes, data=df1, run=False, popdict=popdict)
namesim=''
# run multiple sims to calculate confidential interval
run_msim_conf(sim=sim, to_plot=['new_diagnoses'], save=True, namemsim=namesim)

#### Нужна ли новая калибровка

In [ ]:
previous_cal_day='2021-01-17' #день начала калибровки
timedelt = 40 # период рекалибровки
previous_cal_day, day_today, run_calibration=if_new_calib(previous_cal_day=previous_cal_day, timedelt=timedelt)

if run_calibration:
    sim,p = calibration_process(param=p, datafile=df1, location=location, pop_location=pop_location, start_day=start_day,
                              end_day=day_today, cal_keys=cal_keys, cal_values=cal_values, pop_inf=pop_inf,
                              school_days=school_days, school_changes=school_changes)
    sim=run_model(p=p, end_day=day_today,location=location, pop_location=pop_location, start_day=start_day, school_days=school_days, school_changes=school_changes, 
                 b_days=b_days, b_changes=b_changes, data=df1,run=True)
else: 
    sim=run_model(p=p, end_day=day_today,location=location, pop_location=pop_location, start_day=start_day, school_days=school_days, school_changes=school_changes, 
                 b_days=b_days, b_changes=b_changes, data=df1,run=True)
    
#sim.plot(to_plot=['new_diagnoses','new_infections'],show_args={'interventions':False})

#### Доверительный интервал

In [ ]:
sim=run_model(p=p, location=location, pop_location=pop_location, start_day=start_day, end_day=last_day, 
              b_days=b_days, b_changes=b_changes, data=df1, run=False, popdict=popdict)
namesim='Chelyabinsk_25'
# run multiple sims to calculate confidential interval
run_msim_conf(sim=sim, to_plot=['new_diagnoses'], save=True, namemsim=namesim)

In [ ]:
# detailed plot of confidential interval
msi_path = "" # путь до файла с симуляцией
msi=cv.load(msi_path) 
dday=80
test_data=None


msi.plot(to_plot=['new_diagnoses'],fig_args={'figsize': figsize},test_data=test_data)
msi.plot(to_plot=['new_diagnoses'],fig_args={'figsize': figsize}, dday=dday, test_data=test_data, dateformat=dateformat)
msi.plot(to_plot=['new_deaths'],fig_args={'figsize': figsize},test_data=test_data)
msi.plot(to_plot=['new_deaths'],fig_args={'figsize': figsize},dday=dday, test_data=test_data, dateformat=dateformat)
msi.plot(to_plot=['new_recoveries'],fig_args={'figsize': figsize}, test_data=test_data)
msi.plot(to_plot=['new_recoveries'],fig_args={'figsize': figsize}, dday=dday, test_data=test_data, dateformat=dateformat)
msi.plot(to_plot=['n_critical'],fig_args={'figsize': figsize}, test_data=test_data)
msi.plot(to_plot=['n_critical'],fig_args={'figsize': figsize}, dday=dday, test_data=test_data, dateformat=dateformat)

### Моделирование

In [ ]:
sim.run()

In [ ]:
fit = sim.compute_fit()
fit.summarize()

In [ ]:
sim.plot(to_plot=['new_infections','cum_infections', 'n_infectious'])

In [ ]:
fit.plot()
fit.summarize()

### Параметры популяции

In [ ]:
sim.people.plot()